In [2]:
require(ggplot2)
require(data.table)
require(tidyverse)

In [3]:
sample <- "RMM_308_1_2_3"

In [4]:
loop_stats <- fread("./cicero_res_v2/beta_loops_log10_anchorDist_stats.csv") %>% 
    filter(loop_set != "cicero_m05_05" | grepl("m1|m2", loop_set))

In [5]:
getCenter <- function(loop_file, res = 10000) {
    (apa_file <- paste0(sample, "/apa/", loop_file, "/", res, "/gw/rankAPA.txt"))
    if (file.exists(apa_file)) {
        return(fread(cmd = paste0("cat ", apa_file, "|sed 's/\\]//g;s/\\[//g'"))[[11, 
            11]])
    } else {
        return(NA)
    }
    
}

calcAPA <- function(loop_file, res = 10000, corner_size = 6, type = "rankAPA") {
    input_res <- paste0(sample, "/apa/", loop_file, "/", res, "/gw/", type, ".txt")
    if (file.exists(input_res)) {
        input_res_apa <- fread(cmd = paste0("cat ", input_res, "|sed 's/\\]//g;s/\\[//g'"))
        
        dat <- c(input_res_apa[[11, 11]], input_res_apa[(21 - corner_size + 1):21, 
            1:corner_size] %>% as.matrix %>% as.vector)
        # zscore <- (dat[1] - mean(dat[-1]))/sd(dat[-1])
        apa <- dat[1]/mean(dat[-1])
        return(apa)
        # return(data.frame(APA = apa, Zscore = zscore))
        
    } else {
        return(NA)
    }  #return(data.frame(APA = NA, Zscore = NA)))
}
getAPApng <- function(idx, res = 10000, corner_size = 6, type = "rankAPA") {
    apa_file <- paste0(sample, "/apa/", loop_stats$loop_file[idx], "/", res, "/gw/", 
        type, ".png")
    names(apa_file) <- loop_stats$loop_set[idx]
    if (file.exists(apa_file)) {
        return(apa_file)
    } else {
        return(NA)
    }
}

In [10]:
loop_stats<- loop_stats %>%filter(!grepl('m1|m2',loop_set))
loop_stats<- loop_stats%>%select(loop_set,loop_file,center_percentle)
loop_stats$rankAPA_r10k <- sapply(loop_stats$loop_file, calcAPA)
loop_stats$rankAPA_r5k <- sapply(loop_stats$loop_file, calcAPA, res = 5000)
loop_stats$APA_r10k <- sapply(loop_stats$loop_file, calcAPA,type='APA')
loop_stats$APA_r5k <- sapply(loop_stats$loop_file, calcAPA, res = 5000,type='APA')

loop_stats$center_percentle_r10k <- sapply(loop_stats$loop_file, getCenter)
loop_stats$center_percentle_r5k <- sapply(loop_stats$loop_file, getCenter, res = 5000)
loop_stats %>% select(-loop_file, -center_percentle)

loop_set,rankAPA_r10k,rankAPA_r5k,APA_r10k,APA_r5k,center_percentle_r10k,center_percentle_r5k
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
cicero_05_10,0.8026006,0.9725139,0.6983411,0.8527647,55.65331,51.45448
cicero_10_15,0.8176788,0.9970264,0.7083956,0.8718841,56.71751,52.67704
cicero_15_20,0.8261933,1.0012233,0.7130674,0.8731137,57.45793,53.29851
cicero_gt_20,0.8295752,1.0024924,0.7221319,0.8828889,57.21092,52.91346
cicero_m05_05.r100000,0.7625917,0.9022093,0.6872435,0.8098058,51.89385,46.25398
Islet_HiC,1.6463838,1.6606024,2.7727761,2.2598343,92.94469,80.68784
pcHiC,1.2986967,1.2054302,1.3966691,1.1876392,81.60465,65.09227


In [19]:
glucose_high_apas <- sapply(1:nrow(loop_stats), getAPApng)
glucose_high_apas_5k <- sapply(1:nrow(loop_stats), getAPApng, res = 5000)
require(htmltools)
thumbnail <- function(title = "", img, caption = TRUE, colsize = "col-lg-2") {
    div(class = colsize, p(a(href = sub("png", "txt", img), title), align = "center"), 
        a(herf = "#", class = "thumbnail", img(src = img, class = "img-fluid")))
}
tglist <- tagList(tags$link(rel = "stylesheet", href = "https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0-alpha.6/css/bootstrap.min.css", 
    integrity = "sha384-rwoIResjU2yc3z8GV/NPeZWAv56rSmLldC3R/AZzGRnGxQQKnKkoFVhFQhNUwEyJ", 
    crossorigin = "anonymous"), div(class = "row", div(class = "col-lg", h3("10k resolution rankAPA", 
    align = "center"))), p(), tags$div(class = "row", lapply(c(1:5, 7), function(i) (thumbnail(names(glucose_high_apas)[i], 
    glucose_high_apas[i]))) %>% tagList), hr())

tglist2 <- tagList(div(class = "row", div(class = "col-lg", h3("5k resolution rankAPA", 
    align = "center"))), p(), tags$div(class = "row", lapply(c(1:5, 7), function(i) (thumbnail(names(glucose_high_apas_5k)[i], 
    glucose_high_apas_5k[i]))) %>% tagList), hr())

save_html(c(tglist, tglist2), "rankAPA_compare_res.html")

In [20]:
glucose_high_apas <- sapply(1:nrow(loop_stats), getAPApng, type = "APA")
glucose_high_apas_5k <- sapply(1:nrow(loop_stats), getAPApng, type = "APA", res = 5000)
tglist <- tagList(tags$link(rel = "stylesheet", href = "https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0-alpha.6/css/bootstrap.min.css", 
    integrity = "sha384-rwoIResjU2yc3z8GV/NPeZWAv56rSmLldC3R/AZzGRnGxQQKnKkoFVhFQhNUwEyJ", 
    crossorigin = "anonymous"), div(class = "row", div(class = "col-lg", h3("10k resolution rankAPA", 
    align = "center"))), p(), tags$div(class = "row", lapply(c(1:5, 7), function(i) (thumbnail(names(glucose_high_apas)[i], 
    glucose_high_apas[i]))) %>% tagList), hr())

tglist2 <- tagList(div(class = "row", div(class = "col-lg", h3("5k resolution rankAPA", 
    align = "center"))), p(), tags$div(class = "row", lapply(c(1:5, 7), function(i) (thumbnail(names(glucose_high_apas_5k)[i], 
    glucose_high_apas_5k[i]))) %>% tagList), hr())

save_html(c(tglist, tglist2), "APA_compare_res.html")